Recognizing that it is not trivial to implement all pandas feature in cuDF, NVIDIA decided to add a compatibility layer that will execute pandas operations on GPU if possible, and fall back on the CPU (pandas) implementation when not.

To this end it suffices to load the `cudf.pandas` extension.

In [1]:
%load_ext cudf.pandas
import pandas as pd
import numpy as np

## Creating dataframes

We create a dataframe to do some testing.

In [2]:
nr_rows = 20_000_000
data = {
    'A': np.random.uniform(0.0, 1.0, size=nr_rows),
    'B': np.random.uniform(0.0, 1.0, size=nr_rows),
    'C': np.random.uniform(0.0, 1.0, size=nr_rows),
}

In [3]:
df = pd.DataFrame(data)

In [4]:
df.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 20000000 entries, 0 to 19999999
Data columns (total 3 columns):
 #   Column  Dtype
---  ------  -----
 0   A       float64
 1   B       float64
 2   C       float64
dtypes: float64(3)
memory usage: 457.8 MB


## Applying functions

Computing the columnwise average shows that the performance is on par with cuDF, so clearly, this is executed on the GPU.

In [5]:
%timeit df.A.mean()

1.02 ms ± 47.5 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


The rowwise mean is of course slower, but again, it is executed on the GPU as well.

In [6]:
%timeit df.mean(axis=1)

42.7 ms ± 2.48 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Creating new columns

Adding a column based on a per-column cmoputation.

In [7]:
%timeit df['D'] = 2.1*df.A + 3.5*df.B

4.92 ms ± 224 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Categorical data

A column of categorical data can be created using the `cut` function.

In [8]:
%timeit pd.cut(df.A, [0.0, 0.2, 0.4, 0.6, 0.8, 1.0], \
               labels=['c1', 'c2', 'c3', 'c4', 'c5'])

3.22 ms ± 681 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
df['label'] = pd.cut(df.A, [0.0, 0.2, 0.4, 0.6, 0.8, 1.0],
                     labels=['c1', 'c2', 'c3', 'c4', 'c5'])

## Group-by

We can apply the `groupby` to the column of labels.

In [10]:
%timeit df[['label', 'A']].groupby('label').mean()

17.9 ms ± 575 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


`%timeit` and `pivot_table` don't seem to get along, so you should use `%time` instead.

In [11]:
%time pd.pivot_table(df, index='label', \
                     values=['A', 'B', 'C'], \
                     aggfunc='mean', \
                     observed=False)

CPU times: user 696 ms, sys: 1.28 s, total: 1.98 s
Wall time: 1.98 s


,A,B,C
label,,,
c1,0.100061,0.500052,0.499819
c2,0.299978,0.500099,0.499959
c3,0.499983,0.499851,0.500068
c4,0.699979,0.500268,0.499775
c5,0.900013,0.500117,0.499966
